In [9]:
import pandas as pd
import numpy as np
import random
import collections
import string
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

In [10]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [11]:
brand_df = pd.read_csv("Brand_db.csv")
lower_df = pd.read_csv("Lower.csv")
middle_df = pd.read_csv("Middle.csv")
upper_df = pd.read_csv("Upper.csv")
category_df = pd.read_csv("category_list.csv")

def convert_int(x):
    return int(x)
category_df["Lower_min"] = category_df["Lower_min"].apply(convert_int)
category_df["Middle_min"] = category_df["Middle_min"].apply(convert_int)
category_df["Upper_min"] = category_df["Upper_min"].apply(convert_int)
category_df["Lower_max"] = category_df["Lower_max"].apply(convert_int)
category_df["Middle_max"] = category_df["Middle_max"].apply(convert_int)
category_df["Upper_max"] = category_df["Upper_max"].apply(convert_int)

In [12]:
list_merchants = ["UPI", "VPS", "ATM", "ACH", "MMT", "IIN", "IPS", "VIN", "CMS", "BIL", "NFS", "OTHER"]
dict_no_fields_narr = {"UPI": 5, "VPS": 5, "ATM": 4, "ACH": 4, "MMT": 6, "IIN": 5, "IPS": 5, "VIN": 4, "CMS": 3, "BIL": 5, "NFS": 4, "OTHER": 1}
already_list,already_list_alpha = [],[]
category_list = ["Bills", "Entertainment", "Foods and Drinks", "Fuel", "Groceries","Personal Care", "Shopping", "Travel"]
category_list_lower = ["Bills", "Foods and Drinks", "Fuel", "Groceries", "Personal Care", "Shopping",  "Travel"]
sub_category_list = brand_df["Sub_category"].unique().tolist()
to_rem = ['Pharmacy', 'Payment App', 'Fuel', 'Electricity', 'Wifi', 'Courier', 'Other', 'Hotel' ,'Spa Brand', 'Insurance', 'Cash Withdrawal' ,'Credit Card', 'Transfer', 'Investment', 'Loan (EMI)', 'Rent']
sub_list = sub_category_list.copy()
for i in to_rem:
    sub_list.remove(i)
sub_category_list = sorted(sub_category_list)

#Sub_list: contains only the sub categories on which it is suitable to apply the model 
sub_list = sorted(sub_list)

#counts = brand_df["Sub_category"].value_counts().tolist()
cnt = brand_df.groupby(['Sub_category']).size().tolist()
sub_df = pd.DataFrame({"sub" : sorted(brand_df["Sub_category"].unique().tolist()), "Count":cnt})
#print(brand_df["Sub_category"].value_counts())
#print(brand_df.groupby(['Sub_category']).size())
#print(brand_df["Sub_category"].unique().tolist())
#sub_category_df
#print(len(brand_df["Sub_category"].unique().tolist()))
#len(counts)
sub_df

print(sub_list)

['Accessory Brand', 'Bag Brand', 'Bar', 'Cab Brand', 'Cinema', 'Cinema Booking Online', 'Clothing Center', 'Coffee Shop', 'Cosmetics', 'E-Pharmacy', 'Electronic Brand', 'Electronic Center', 'Electronic Store', 'Eyewear Brand', 'F Clothing Brand', 'F Footwear Brand', 'Fast Food', 'Food Order', 'Furniture', 'Game Center', 'Game Shop', 'Gym', 'Holiday Planner', 'Hospital', 'Jewellery', 'Liquor', 'M Clothing Brand', 'M Footwear Brand', 'Phone Bill', 'Restaurant', 'Salon', 'Shopping Center', 'Shopping Center Online', 'Small Grocery Shop', 'Snacks', 'Sports Brand', 'Streaming Brand', 'Supermart', 'Supermart Online', 'TV', 'Toy Center', 'U Clothing Brand', 'U Footwear Brand', 'Vehicle Rental', 'Watch Brand']


In [13]:
#Adding sub categories to social class df
b_l = lower_df["Brand_name"].tolist()
b_m = middle_df["Brand_name"].tolist()
b_u = upper_df["Brand_name"].tolist()

s_l, s_m, s_u = [], [], []
for i in b_l:
    index = brand_df.index[brand_df['Brand_name']==i].tolist()
    sub = brand_df.at[index[0], "Sub_category"] 
    s_l.append(sub)
for i in b_m:
    index = brand_df.index[brand_df['Brand_name']==i].tolist()
    sub = brand_df.at[index[0], "Sub_category"] 
    s_m.append(sub)
for i in b_u:
    index = brand_df.index[brand_df['Brand_name']==i].tolist()
    sub = brand_df.at[index[0], "Sub_category"] 
    s_u.append(sub)

lower_df["Sub_category"] = s_l
middle_df["Sub_category"] = s_m
upper_df["Sub_category"] = s_u

In [14]:
brand_df

,Id,Brand_name,Category,Sub_category
0,0,Airtel (Bharti Air),Bills,Phone Bill
1,1,Jio,Bills,Phone Bill
2,2,Amul,Foods and Drinks,Snacks
3,3,Spencers,Groceries,Supermart
4,4,Reliance Fresh,Groceries,Supermart
5,5,Kwality Bazaar,Groceries,Small Grocery Shop
6,6,Shopper Stop,Shopping,Clothing Center
7,7,Reliance Digital,Shopping,Electronic Center
8,8,Louis Philippe,Shopping,M Clothing Brand
9,9,Arrow,Shopping,M Clothing Brand


In [15]:
lower_df.head()

,Brand_name,Category,Min,Max,Sub_category
0,Airtel (Bharti Air),Bills,21,300,Phone Bill
1,Jio,Bills,21,300,Phone Bill
2,Amul,Foods and Drinks,50,100,Snacks
3,Kwality Bazaar,Groceries,50,300,Small Grocery Shop
4,Apollo Pharmacy,Health,50,300,Pharmacy


In [16]:
middle_df.head()

,Brand_name,Category,Min,Max,Sub_category
0,Airtel (Bharti Air),Bills,21,400,Phone Bill
1,Jio,Bills,21,400,Phone Bill
2,Amul,Foods and Drinks,50,300,Snacks
3,Spencers,Groceries,100,2000,Supermart
4,Reliance Fresh,Groceries,100,1500,Supermart


In [17]:
upper_df.head()

,Brand_name,Category,Min,Max,Sub_category
0,Airtel (Bharti Air),Bills,21,2000,Phone Bill
1,Jio,Bills,21,2000,Phone Bill
2,Amul,Foods and Drinks,50,500,Snacks
3,Spencers,Groceries,100,5000,Supermart
4,Reliance Fresh,Groceries,100,5000,Supermart


In [18]:
category_df.head()

,Category,Lower_min,Lower_max,Middle_min,Middle_max,Upper_min,Upper_max
0,Bills,1,2,2,3,3,5
1,Cash Withdrawal,1,3,1,4,1,3
2,Courier,0,0,1,2,1,3
3,Credit Card,0,0,0,1,1,3
4,Entertainment,0,1,2,5,5,10


In [19]:
already_trans_id = []
def gen_trans_id():
    trans_id = ''
    for i in range(8):
        trans_id += random.choice(string.digits)
    trans_id = "M"+trans_id
    if trans_id in already_trans_id:
        trans_id = get_trans_id()
    if trans_id not in already_trans_id:
        already_trans_id.append(trans_id)
    return trans_id    
def gen_gender():
    return random.choice(["M","F"])
def gen_social():
    return random.choice([middle_df, upper_df])

In [20]:
def gen_amount(social_class_df, brand):
    min_amnt = social_class_df.loc[social_class_df['Brand_name']==brand,'Min'].values[0]
    max_amnt = social_class_df.loc[social_class_df['Brand_name']==brand,'Max'].values[0]
    #print(brand, min_amnt, max_amnt)
    return random.randint(min_amnt, max_amnt)    
def gen_no_of_trans(category_df, category, social_class):
    min_no = category_df.loc[category_df['Category'] == category, social_class+"_min"].values[0]
    max_no = category_df.loc[category_df['Category'] == category, social_class+"_max"].values[0]
   # print(min_no, max_no)
    #print(category)
    if max_no == min_no:
        return max_no
    if max_no == 0:
        return 0
    return random.randint(min_no, max_no)

In [21]:
# Generate transactions based on social class (number of transactions per category specified in csv of each social class)
'''
def gen_trans_social(category_df, social_class_df, social_class_name):
    category_df = category_df.copy()
    social_class_df = social_class_df.copy()
    li = []
    l1, l2 = [], []
    li.append(l1)
    li.append(l2)
    if social_class_name=="Lower":
        cat_list = category_list_lower
    else:
        cat_list = category_list
    for c in sub_category_list:
        #print (c)
        n = gen_no_of_trans(category_df, c, social_class_name)
        new_df = social_class_df[social_class_df["Category"] == c ]
        brands_li = new_df["Brand_name"].tolist()
        #print (brands_li)
        for i in range(n):
            brand_name = random.choice(brands_li)
            amnt = gen_amount(social_class_df, brand_name)
            li[0].append(brand_name)
            li[1].append(amnt)
    return li        
'''    

'\ndef gen_trans_social(category_df, social_class_df, social_class_name):\n    category_df = category_df.copy()\n    social_class_df = social_class_df.copy()\n    li = []\n    l1, l2 = [], []\n    li.append(l1)\n    li.append(l2)\n    if social_class_name=="Lower":\n        cat_list = category_list_lower\n    else:\n        cat_list = category_list\n    for c in sub_category_list:\n        #print (c)\n        n = gen_no_of_trans(category_df, c, social_class_name)\n        new_df = social_class_df[social_class_df["Category"] == c ]\n        brands_li = new_df["Brand_name"].tolist()\n        #print (brands_li)\n        for i in range(n):\n            brand_name = random.choice(brands_li)\n            amnt = gen_amount(social_class_df, brand_name)\n            li[0].append(brand_name)\n            li[1].append(amnt)\n    return li        \n'

In [22]:
'''
#For sparse repr method of apriori (alternate if you have less memory)
transactions_lower = gen_trans_social(category_df, lower_df, "Lower")
transactions_middle = gen_trans_social(category_df, middle_df, "Middle")
transactions_upper = gen_trans_social(category_df, upper_df, "Upper")
transactions_list_all = [] 
transactions_list_all.append(transactions_lower[0])
transactions_list_all.append(transactions_middle[0])
transactions_list_all.append(transactions_upper[0])
'''

'\n#For sparse repr method of apriori (alternate if you have less memory)\ntransactions_lower = gen_trans_social(category_df, lower_df, "Lower")\ntransactions_middle = gen_trans_social(category_df, middle_df, "Middle")\ntransactions_upper = gen_trans_social(category_df, upper_df, "Upper")\ntransactions_list_all = [] \ntransactions_list_all.append(transactions_lower[0])\ntransactions_list_all.append(transactions_middle[0])\ntransactions_list_all.append(transactions_upper[0])\n'

In [23]:
#Parent function to generate multiple transactions for different social classes at the same time
'''
def gen_transaction_social(n, category_df, lower_df, middle_df, upper_df):
    transactions = pd.DataFrame()
    for i in range(n):
        #transactions_lower = gen_trans_social(category_df, lower_df, "Lower")
        transactions_middle = gen_trans_social(category_df, middle_df, "Middle")
        transactions_upper = gen_trans_social(category_df, upper_df, "Upper")
        trans_dict_lower = {"Brand":transactions_lower[0] , "Amount":transactions_lower[1]}
        transactions_lower_df = pd.DataFrame(trans_dict_lower)
        trans_id_lower = gen_trans_id()
        transactions_lower_df["txnId"] = trans_id_lower
        transactions_lower_df["Gender"] = gen_gender()
        transactions_lower_df = transactions_lower_df[["txnId", "Brand", "Amount"]]
        
        trans_dict_middle = {"Brand":transactions_middle[0] , "Amount":transactions_middle[1]}
        transactions_middle_df = pd.DataFrame(trans_dict_middle)
        trans_id_middle = gen_trans_id()
        transactions_middle_df["txnId"] = trans_id_middle
        transactions_middle_df["Gender"] = gen_gender()
        transactions_middle_df = transactions_middle_df[["txnId", "Brand", "Amount"]]


        trans_dict_upper = {"Brand":transactions_upper[0] , "Amount":transactions_upper[1]}
        transactions_upper_df = pd.DataFrame(trans_dict_upper)
        trans_id_upper = gen_trans_id()
        transactions_upper_df["txnId"] = trans_id_upper
        transactions_upper_df["Gender"] = gen_gender()
        transactions_upper_df = transactions_upper_df[["txnId", "Brand", "Amount"]]
        
        transactions_new = pd.concat([ transactions_middle_df, transactions_upper_df], axis = 0, ignore_index= True)
        transactions = pd.concat([transactions, transactions_new], axis = 0, ignore_index= True)
    return transactions
'''

'\ndef gen_transaction_social(n, category_df, lower_df, middle_df, upper_df):\n    transactions = pd.DataFrame()\n    for i in range(n):\n        #transactions_lower = gen_trans_social(category_df, lower_df, "Lower")\n        transactions_middle = gen_trans_social(category_df, middle_df, "Middle")\n        transactions_upper = gen_trans_social(category_df, upper_df, "Upper")\n        trans_dict_lower = {"Brand":transactions_lower[0] , "Amount":transactions_lower[1]}\n        transactions_lower_df = pd.DataFrame(trans_dict_lower)\n        trans_id_lower = gen_trans_id()\n        transactions_lower_df["txnId"] = trans_id_lower\n        transactions_lower_df["Gender"] = gen_gender()\n        transactions_lower_df = transactions_lower_df[["txnId", "Brand", "Amount"]]\n        \n        trans_dict_middle = {"Brand":transactions_middle[0] , "Amount":transactions_middle[1]}\n        transactions_middle_df = pd.DataFrame(trans_dict_middle)\n        trans_id_middle = gen_trans_id()\n        tra

In [24]:
#transactions = gen_transaction_social(50, category_df, lower_df, middle_df, upper_df)

In [25]:
print(sub_list)
#print(sub_df)

['Accessory Brand', 'Bag Brand', 'Bar', 'Cab Brand', 'Cinema', 'Cinema Booking Online', 'Clothing Center', 'Coffee Shop', 'Cosmetics', 'E-Pharmacy', 'Electronic Brand', 'Electronic Center', 'Electronic Store', 'Eyewear Brand', 'F Clothing Brand', 'F Footwear Brand', 'Fast Food', 'Food Order', 'Furniture', 'Game Center', 'Game Shop', 'Gym', 'Holiday Planner', 'Hospital', 'Jewellery', 'Liquor', 'M Clothing Brand', 'M Footwear Brand', 'Phone Bill', 'Restaurant', 'Salon', 'Shopping Center', 'Shopping Center Online', 'Small Grocery Shop', 'Snacks', 'Sports Brand', 'Streaming Brand', 'Supermart', 'Supermart Online', 'TV', 'Toy Center', 'U Clothing Brand', 'U Footwear Brand', 'Vehicle Rental', 'Watch Brand']


In [26]:
#Using sub_list: contains only the sub categories on which it is suitable to apply the model 
#sub_dict denotes the maximum number of brands of a specific sub-category in the spendings of a user
sub_dict = {'Accessory Brand' : 1,
 'Bag Brand' : 1 ,
 'Bar' : 2,
 'Cab Brand' : 2,
 'Cinema': 2,
 'Cinema Booking Online' : 1,
 'Clothing Center' : 2,
 'Coffee Shop' : 1,
 'Cosmetics' : 2,
 'E-Pharmacy': 1,
 'Electronic Brand': 1,
 'Electronic Center': 2,
 'Electronic Store': 1,
 'Eyewear Brand': 1,
 'F Clothing Brand': 3,
 'F Footwear Brand': 1,
 'Fast Food': 3,
 'Food Order': 2,
 'Furniture': 1,
 'Game Center': 1,
 'Game Shop': 1,
 'Gym': 1,
 'Holiday Planner': 2,
 'Hospital': 2,
 'Jewellery': 1,
 'Liquor': 1,
 'M Clothing Brand': 3,
 'M Footwear Brand': 1,
 'Phone Bill': 1,
 'Restaurant': 5,
 'Salon': 2,
 'Shopping Center': 1,
 'Shopping Center Online': 2,
 'Small Grocery Shop': 2,
 'Snacks': 5,
 'Sports Brand': 1,
 'Streaming Brand': 2,
 'Supermart': 2,
 'Supermart Online': 1,
 'TV': 1,
 'Toy Center': 1,
 'U Clothing Brand': 3,
 'U Footwear Brand': 1,
 'Vehicle Rental': 1,
 'Watch Brand': 1} 


mini_sub_dict = {'Liquor': 1, 'Jewellery': 1}

In [27]:
def gen_trans_sub():
    gender = gen_gender()
    userId = gen_trans_id()
    social_class_df = gen_social()
    if gender == "M":
        ignore_keys = ["F Clothing Brand", "F Footwear Brand", "Cosmetics", "Jewellery"]
    else:
        ignore_keys = ["M Clothing Brand", "M Footwear Brand"]

    set_of_brands = []
    #print("Gender: ",gender)
    for sub in mini_sub_dict.keys():
        if sub not in ignore_keys:
            sub_freq = sub_dict[sub]
            sub_freq = random.randint(0,sub_freq)
          #  print("sub: ", sub, "sub_freq: ", sub_freq)
            if sub_freq == 0:
                continue
            else:
                index = social_class_df.index[social_class_df['Sub_category'] == sub ].tolist()
                n = len(index)
                possible_brands = [social_class_df.at[index[i], "Brand_name"] for i in range(n)]
                selected_brands = random.sample(possible_brands, sub_freq)         
                set_of_brands += selected_brands
    return (userId, gender, social_class_df, set_of_brands)
                
                
        

In [28]:
userId, gender, social_class_df, set_of_brands  = gen_trans_sub()
print(userId, gender, set_of_brands)

M33955518 M []


In [29]:
def gen_multi_user(n):
    transactions = pd.DataFrame()
    for i in range(n):
        userId, gender, social_class_df, set_of_brands  = gen_trans_sub()
        trans_dict = {"Brand":set_of_brands, "Amount":1}
        transactions_df = pd.DataFrame(trans_dict)
        transactions_df["userId"] = userId
        transactions_df["Gender"] = gender
        transactions_df = transactions_df[["userId", "Gender", "Brand", "Amount"]]
        transactions = pd.concat([transactions, transactions_df], axis = 0, ignore_index= True)
    
    return transactions
        

In [36]:
transactions = gen_multi_user(1)


try_brands1 = ["Zara", "H&M", "KFC" , "Samsung", "Vogo", "Burger King" ]#, "Casio", "Amul", "Dunkin Donuts"]
try_brands2 = ["Zara", "Levis", "KFC" ,"Croma", "Samsung", "Vogo", "Burger King" ]#, "Casio", "CEX","Nike","Fitbit","Lenovo"]
try_brands3 = ["Zara", "H&M", "KFC" , "Croma", "Vogo", "Casio" ]# "Amul", "Dunkin Donuts", "Nykaa", "Manyavar"]
try_brands4 = ["Zara", "H&M", "SK Rolls" , "Burger King", "Casio"] #, "Amul", "Dunkin Donuts","Fitbit","Lenovo" ]
try_brands5 = ["Zara", "H&M", "Red Tape" , "SK Rolls", "Bounce", "Casio"] # "Amul", "Dunkin Donuts", "Lenovo"]
df1 = pd.DataFrame({"Brand":try_brands1, "Amount":1 })
df2 = pd.DataFrame({"Brand":try_brands2, "Amount":1 })
df3 = pd.DataFrame({"Brand":try_brands3, "Amount":1 })
df4 = pd.DataFrame({"Brand":try_brands4, "Amount":1 })
df5 = pd.DataFrame({"Brand":try_brands5, "Amount":1 })
user_id = ["01", "02", "03", "04", "05"]
df1["userId"] = user_id[0]
df2["userId"] = user_id[1]
df3["userId"] = user_id[2]
df4["userId"] = user_id[3]
df5["userId"] = user_id[4]
transactions = pd.concat([df1, df2, df3, df4, df5], axis = 0, ignore_index= True)

In [37]:
transactions[transactions["Brand"]=="Zara"].shape

(5, 3)

In [38]:
basket = pd.pivot_table(data=transactions, index='userId', columns='Brand',values='Amount', aggfunc='sum',fill_value=0)

In [39]:
#basket.head()

In [40]:
def convert_into_binary(x):
    if x > 0:
        return 1
    else:
        return 0

In [41]:
basket_sets = basket.applymap(convert_into_binary)
basket_sets.head()

Brand,Bounce,Burger King,Casio,Croma,H&M,KFC,Levis,Red Tape,SK Rolls,Samsung,Vogo,Zara
userId,,,,,,,,,,,,
01,0,1,0,0,1,1,0,0,0,1,1,1
02,0,1,0,1,0,1,1,0,0,1,1,1
03,0,0,1,1,1,1,0,0,0,0,1,1
04,0,1,1,0,1,0,0,0,1,0,0,1
05,1,0,1,0,1,0,0,1,1,0,0,1


In [42]:
frequent_itemsets = apriori(basket_sets, min_support=0.1, use_colnames=True, low_memory= True)
#oht_ary = te.fit(basket_sets).transform(dataset, sparse=True)
#sparse_df = pd.DataFrame.sparse.from_spmatrix(oht_ary, columns=te.columns_)
#sparse_df

In [43]:
frequent_itemsets.head()

,support,itemsets
0,0.2,(Bounce)
1,0.6,(Burger King)
2,0.6,(Casio)
3,0.4,(Croma)
4,0.8,(H&M)


In [44]:
rules_mlxtend = association_rules(frequent_itemsets, metric="lift")
rules_mlxtend.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Bounce),(Casio),0.2,0.6,0.2,1.000000,1.666667,0.08,inf
1,(Casio),(Bounce),0.6,0.2,0.2,0.333333,1.666667,0.08,1.200000
2,(H&M),(Bounce),0.8,0.2,0.2,0.250000,1.250000,0.04,1.066667
3,(Bounce),(H&M),0.2,0.8,0.2,1.000000,1.250000,0.04,inf
4,(Bounce),(Red Tape),0.2,0.2,0.2,1.000000,5.000000,0.16,inf


In [45]:
rules_mlxtend[ (rules_mlxtend['lift'] >= 5) & (rules_mlxtend['confidence'] >= 0.8) ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
4,(Bounce),(Red Tape),0.2,0.2,0.2,1.0,5.0,0.16,inf
5,(Red Tape),(Bounce),0.2,0.2,0.2,1.0,5.0,0.16,inf
89,"(Bounce, Casio)",(Red Tape),0.2,0.2,0.2,1.0,5.0,0.16,inf
90,"(Red Tape, Casio)",(Bounce),0.2,0.2,0.2,1.0,5.0,0.16,inf
91,(Bounce),"(Red Tape, Casio)",0.2,0.2,0.2,1.0,5.0,0.16,inf
92,(Red Tape),"(Bounce, Casio)",0.2,0.2,0.2,1.0,5.0,0.16,inf
106,"(H&M, Bounce)",(Red Tape),0.2,0.2,0.2,1.0,5.0,0.16,inf
107,"(H&M, Red Tape)",(Bounce),0.2,0.2,0.2,1.0,5.0,0.16,inf
110,(Bounce),"(H&M, Red Tape)",0.2,0.2,0.2,1.0,5.0,0.16,inf
111,(Red Tape),"(H&M, Bounce)",0.2,0.2,0.2,1.0,5.0,0.16,inf


In [ ]:
te = TransactionEncoder()
oht_ary = te.fit(transactions_list_all).transform(transactions_list_all, sparse=True)
sparse_df = pd.DataFrame.sparse.from_spmatrix(oht_ary, columns=te.columns_)
sparse_df

In [ ]:
apriori(sparse_df, min_support=0.6, use_colnames=True, verbose=1)